In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy import vstack
from numpy.random import randn
from numpy.random import randint
#from keras.datasets.cifar10 import load_data
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
from matplotlib import pyplot
from keras.initializers import RandomNormal
import os
from numpy import asarray
from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
from PIL import Image
 

In [ ]:
# define the standalone discriminator model
def define_discriminator(in_shape=(64,64,3)):
	model = Sequential()
	# normal
	model.add(Conv2D(64, kernel_size=3, padding='same', input_shape=in_shape))
	model.add(LeakyReLU(alpha=0.2))
	# downsample
	model.add(Conv2D(128,  kernel_size=3, strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# downsample
	model.add(Conv2D(128,  kernel_size=3, strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# downsample
	model.add(Conv2D(256,  kernel_size=3, strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# classifier
	model.add(Flatten())
	model.add(Dense(1, activation='sigmoid'))
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
	return model

In [ ]:
define_discriminator().summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 64, 64, 64)        1792      
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 64, 64, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 128)       73856     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 32, 32, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 128)       147584    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 256)         2

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
# define the standalone generator model
def define_generator(latent_dim):
  model = Sequential()
  init = RandomNormal(stddev=0.02)
  
	# foundation for 4x4 image
  n_nodes = 256 * 4 * 4
  model.add(Dense(n_nodes, input_dim=latent_dim))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Reshape((4, 4, 256)))
	# upsample to 8x8
  model.add(Conv2DTranspose(128, kernel_size=3, strides=2, padding='same'))
  model.add(LeakyReLU(alpha=0.2))
	# upsample to 16x16
  model.add(Conv2DTranspose(128, kernel_size=3, strides=2, padding='same'))
  model.add(LeakyReLU(alpha=0.2))
	# upsample to 32x32
  model.add(Conv2DTranspose(128, kernel_size=3, strides=2, padding='same'))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Conv2DTranspose(128,kernel_size=3, strides=2, padding='same', kernel_initializer=init))
  model.add(LeakyReLU(alpha=0.2))
	# output layer
  model.add(Conv2D(3, kernel_size=3, activation='tanh', padding='same'))
  return model
  

In [ ]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
	# make weights in the discriminator not trainable
	d_model.trainable = False
	# connect them
	model = Sequential()
	# add generator
	model.add(g_model)
	# add the discriminator
	model.add(d_model)
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt)
	return model

In [ ]:
# load and prepare cifar10 training images
def load_real_samples():
  os.chdir("/content/drive/MyDrive/All_Images")
  paths=pd.read_csv('/content/drive/MyDrive/All_Images/glaucoma.csv')
  dataset=[]
  for i in range(650):
    if(os.path.exists(paths.iloc[i][0])):
      im = Image.open(paths.iloc[i][0])
      #print(im)
      resized_im = im.resize((round(im.size[0]*0.02083), round(im.size[1]*0.03125)))
      #print(resized_im)
      image=asarray(resized_im)
      data = asarray(image)
      dataset.append(data)
  trainX=np.array(dataset)
	# load cifar10 dataset
	# (trainX, _), (_, _) = load_data()
	# convert from unsigned ints to floats
  X = trainX.astype('float32')
	# scale from [0,255] to [-1,1]
  X = (X - 127.5) / 127.5
  return X
#a=load_real_samples()
#print(a)

In [ ]:
# select real samples
def generate_real_samples(dataset, n_samples):
	# choose random instances
	ix = randint(0, dataset.shape[0], n_samples)
	# retrieve selected images
	X = dataset[ix]
	# generate 'real' class labels (1)
	y = ones((n_samples, 1))
	return X, y

In [ ]:
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
	# generate points in the latent space
	x_input = randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the network
	x_input = x_input.reshape(n_samples, latent_dim)
	return x_input

In [ ]:
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(g_model, latent_dim, n_samples):
	# generate points in latent space
	x_input = generate_latent_points(latent_dim, n_samples)
	# predict outputs
	X = g_model.predict(x_input)
	# create 'fake' class labels (0)
	y = zeros((n_samples, 1))
	return X, y

In [ ]:
# create and save a plot of generated images
def save_plot(examples, epoch, n=7):
	# scale from [-1,1] to [0,1]
	examples = (examples + 1) / 2.0
	# plot images
	for i in range(n * n):
		# define subplot
		pyplot.subplot(n, n, 1 + i)
		# turn off axis
		pyplot.axis('off')
		# plot raw pixel data
		pyplot.imshow(examples[i])
	# save plot to file
	filename = 'generated_plot_e%03d.png' % (epoch+1)
	pyplot.savefig(filename)
	pyplot.close()
 

In [ ]:
# evaluate the discriminator, plot generated images, save generator model
def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, n_samples=650):
	# prepare real samples
	X_real, y_real = generate_real_samples(dataset, n_samples)
	# evaluate discriminator on real examples
	_, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
	# prepare fake examples
	x_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
	# evaluate discriminator on fake examples
	_, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
	# summarize discriminator performance
	print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))
	# save plot
	save_plot(x_fake, epoch)
	# save the generator model tile file
	filename = 'generator_model_%03d.h5' % (epoch+1)
	g_model.save(filename)

In [ ]:
# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=200, n_batch=64):
	bat_per_epo = int(dataset.shape[0] / n_batch)
	half_batch = int(n_batch / 2)
	# manually enumerate epochs
	for i in range(n_epochs):
		# enumerate batches over the training set
		for j in range(bat_per_epo):
			# get randomly selected 'real' samples
			X_real, y_real = generate_real_samples(dataset, half_batch)
			# update discriminator model weights
			d_loss1, _ = d_model.train_on_batch(X_real, y_real)
			# generate 'fake' examples
			X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
			# update discriminator model weights
			d_loss2, _ = d_model.train_on_batch(X_fake, y_fake)
			# prepare points in latent space as input for the generator
			X_gan = generate_latent_points(latent_dim, n_batch)
			# create inverted labels for the fake samples
			y_gan = ones((n_batch, 1))
			# update the generator via the discriminator's error
			g_loss = gan_model.train_on_batch(X_gan, y_gan)
			# summarize loss on this batch
			print('>%d, %d/%d, d1=%.3f, d2=%.3f g=%.3f' %
				(i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss))
		# evaluate the model performance, sometimes
		if (i+1) % 10 == 0:
			summarize_performance(i, g_model, d_model, dataset, latent_dim)

In [ ]:
# size of the latent space
latent_dim = 100
# create the discriminator
d_model = define_discriminator()
# create the generator
g_model = define_generator(latent_dim)
# create the gan
gan_model = define_gan(g_model, d_model)
# load image data
dataset = load_real_samples()
# train model
train(g_model, d_model, gan_model, dataset, latent_dim)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


>1, 1/10, d1=0.679, d2=0.695 g=0.692
>1, 2/10, d1=0.501, d2=0.695 g=0.692
>1, 3/10, d1=0.349, d2=0.697 g=0.691
>1, 4/10, d1=0.189, d2=0.700 g=0.690
>1, 5/10, d1=0.077, d2=0.706 g=0.687
>1, 6/10, d1=0.031, d2=0.715 g=0.682
>1, 7/10, d1=0.015, d2=0.728 g=0.675
>1, 8/10, d1=0.013, d2=0.747 g=0.669
>1, 9/10, d1=0.016, d2=0.759 g=0.674
>1, 10/10, d1=0.028, d2=0.764 g=0.703
>2, 1/10, d1=0.054, d2=0.744 g=0.752
>2, 2/10, d1=0.078, d2=0.715 g=0.795
>2, 3/10, d1=0.054, d2=0.640 g=0.858
>2, 4/10, d1=0.033, d2=0.580 g=0.922
>2, 5/10, d1=0.016, d2=0.535 g=0.981
>2, 6/10, d1=0.016, d2=0.510 g=1.012
>2, 7/10, d1=0.010, d2=0.487 g=1.057
>2, 8/10, d1=0.005, d2=0.451 g=1.137
>2, 9/10, d1=0.006, d2=0.398 g=1.271
>2, 10/10, d1=0.004, d2=0.328 g=1.487
>3, 1/10, d1=0.006, d2=0.249 g=1.786
>3, 2/10, d1=0.001, d2=0.167 g=2.224
>3, 3/10, d1=0.001, d2=0.100 g=2.748
>3, 4/10, d1=0.002, d2=0.058 g=3.265
>3, 5/10, d1=0.001, d2=0.035 g=3.733
>3, 6/10, d1=0.000, d2=0.023 g=4.146
>3, 7/10, d1=0.000, d2=0.016 g=4.521